## Adult Dataset Part 1: Cleaning and Preparation

#### Objectives:
1. Preprocess the data and store the cleaned dataset as adult_clean.csv
2. Load clean dataset and test supervised/unsupervised models
    - Goal: Determine best model to predict if new entry earns >50k or <50k with supervised learning
    - Goal: Find insights and patterns in data using unspuervised learning
3. Apply k-folds cross validation
    - Goal: Evaluate robustness of results across different models.
    - Goal: Use and justify different error metrics


#### Summary of dataset

Datasource: https://archive.ics.uci.edu/ml/datasets/adult

Feature descrptions copied below from the UCI website:

- age: continuous. 
- workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked. 
- fnlwgt: Continuous. A weighting assigned from the sampling process.
- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool. 
- education-num: continuous. 
- marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse. 
- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces. 
- relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried. 
- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black. 
- sex: Female, Male. 
- capital-gain: continuous.
- capital-loss: continuous. 
- hours-per-week: continuous. 
- native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
- target: >50K, <=50K.

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
#original data does not come with headers. add headers based on the documentation descrption
headers = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 
           'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 
           'hours-per-week', 'native-country', 'target']

In [11]:
data = pd.read_csv('./data/adult.data', names = headers)
df = data.copy()
print(df.isna().sum())

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
target            0
dtype: int64


In [12]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [13]:
df.sample(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
19157,27,Private,145284,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,40,United-States,<=50K
2705,40,Private,218903,HS-grad,9,Divorced,Tech-support,Unmarried,White,Female,0,0,40,United-States,<=50K
2644,19,Private,196857,HS-grad,9,Never-married,Other-service,Not-in-family,White,Female,0,0,40,United-States,<=50K
12487,34,Private,119153,11th,7,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,50,United-States,<=50K
30368,26,Private,187248,HS-grad,9,Married-civ-spouse,Prof-specialty,Not-in-family,Black,Male,0,0,40,United-States,<=50K
15973,36,Private,66304,Some-college,10,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,45,United-States,<=50K
22550,44,Private,171722,HS-grad,9,Separated,Other-service,Unmarried,White,Female,0,0,39,United-States,<=50K
3522,45,Self-emp-not-inc,204205,10th,6,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,65,United-States,<=50K
6992,31,Private,182237,Assoc-acdm,12,Married-civ-spouse,Sales,Husband,White,Male,4386,0,45,United-States,>50K
29645,22,Private,190903,11th,7,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,20,United-States,<=50K


On first look, it appears there are no missing values. **However the dataset descrption indicates that missing values have been flled with '?'.** Figure out what to do here :)

In [18]:
#code
print(df.iloc[11526])

df = df.replace("[?]", value=np.nan, regex=True)
print(df.iloc[11526])
print(df.isna().sum())

age                           18
workclass                    NaN
fnlwgt                     28357
education           Some-college
education-num                 10
marital-status     Never-married
occupation                   NaN
relationship           Own-child
race                       White
sex                       Female
capital-gain                   0
capital-loss                   0
hours-per-week                20
native-country     United-States
target                     <=50K
Name: 11526, dtype: object
age                           18
workclass                    NaN
fnlwgt                     28357
education           Some-college
education-num                 10
marital-status     Never-married
occupation                   NaN
relationship           Own-child
race                       White
sex                       Female
capital-gain                   0
capital-loss                   0
hours-per-week                20
native-country     United-States
target          

Before deciding **how to deal with the missing values want to know how they influence the data**. For example, if workclass and occupation for example both have missing values (in the same tuple) it would be reasonable to drop the data.

In [15]:
occupation_nan = df[df['occupation'].isna()]
workclass_nan = df[df['workclass'].isna()]
print(f"shape of occupation_nan: {occupation_nan.shape}")
print(f"shape of workclass_nan: {workclass_nan.shape}")
nan_intersection_len = len(occupation_nan.index.intersection(workclass_nan.index))
print(f"workclass nan as part of occupation nan: {nan_intersection_len}")

shape of occupation_nan: (1843, 15)
shape of workclass_nan: (1836, 15)
workclass nan as part of occupation nan: 1836


**What % of the dataset do nans represent?** What is the **best approach** for dealing with the nan's?

In [17]:
total_rows =df.shape[0]
total_without_nans = df.dropna().shape[0]
total_nans = total_rows - total_without_nans
percent_nans = round(total_nans / total_rows, 2)
print(f"The percent of Nans is {percent_nans} or {total_nans}/{total_rows}")

print(df.info())

The percent of Nans is 0.07 or 2399/32561
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       30725 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      30718 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  31978 non-null  object
 14  target          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB
None


In [14]:
#plot the categroical columns to see what we are dealing wit
# Add greater_than_50

nn_df = df.dropna()
nn_df.to_csv("data/adult_clean.csv")

In [15]:
#plot the numerical columns


#### Additional Steps:
1. Apply statistical tests to determine if dropping values changes the structure of the distribution in the workplace, occupation, and native-country categories i.e. t-test for differnce of modes. 
2. Compare filling the workplace and occupation columns with mode values and applying same statistical tests.

In [26]:
def one_hot_encode_column(df, col_name):
    unique_splits = df[col_name].unique()
    df[[new_col for new_col in unique_splits]] = np.nan
    for new_col in unique_splits:
        print(new_col)
        df[new_col] = df[col_name].str.contains(new_col)
        df[new_col] = df[new_col].map({True: 1, False: 0})
        
    print(df.sample(10))
    return df

ehc_df = one_hot_encode_column(nn_df,'marital-status')
ehc_df = one_hot_encode_column(ehc_df,'education')
ehc_df = one_hot_encode_column(ehc_df,'workclass')
ehc_df = one_hot_encode_column(ehc_df,'relationship')
ehc_df = one_hot_encode_column(ehc_df,'occupation')
ehc_df = one_hot_encode_column(ehc_df,'sex')
ehc_df = one_hot_encode_column(ehc_df,'race')
ehc_df = one_hot_encode_column(ehc_df,'relationship')
ehc_df = one_hot_encode_column(ehc_df,'native-country')



ehc_df.to_csv("data/adult_clean_ehc.csv")

<ipython-input-26-43cc6d7c5bb3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[[new_col for new_col in unique_splits]] = np.nan
/home/aimwps/anaconda3/envs/AIMwpS/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-26-43cc6d7c5bb3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

 Never-married
 Married-civ-spouse
 Divorced
 Married-spouse-absent
 Separated
 Married-AF-spouse
 Widowed
       age     workclass  fnlwgt      education  education-num  \
8541    42     Local-gov  188291      Bachelors             13   
22867   33   Federal-gov  331615      Doctorate             16   
20330   37       Private  315291      Bachelors             13   
1174    47       Private  183013        HS-grad              9   
15922   23       Private  201680           12th              8   
11126   39       Private  156897        HS-grad              9   
3181    21       Private   57211        HS-grad              9   
12273   23       Private  213719     Assoc-acdm             12   
1445    50       Private   30447      Assoc-voc             11   
6742    21       Private   83033   Some-college             10   

            marital-status         occupation     relationship    race  \
8541    Married-civ-spouse    Protective-serv          Husband   White   
22867   Married-ci

<ipython-input-26-43cc6d7c5bb3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_col] = df[col_name].str.contains(new_col)
<ipython-input-26-43cc6d7c5bb3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_col] = df[new_col].map({True: 1, False: 0})


 HS-grad
 11th
 Masters
 9th
 Some-college
 Assoc-acdm
 7th-8th
 Doctorate
 Assoc-voc
 Prof-school
 5th-6th
 10th
 Preschool
 12th
 1st-4th
       age   workclass  fnlwgt      education  education-num  \
30595   43     Private  342567      Bachelors             13   
11347   20     Private  105585   Some-college             10   
14216   34   State-gov  252529      Bachelors             13   
15067   51   Local-gov   74784   Some-college             10   
12277   24     Private   99970      Bachelors             13   
6365    50   Local-gov  370733        HS-grad              9   
18256   20     Private   58222   Some-college             10   
30089   21     Private  170302        HS-grad              9   
16866   48     Private   36228        HS-grad              9   
24987   57   State-gov  283635   Some-college             10   

               marital-status          occupation     relationship    race  \
30595   Married-spouse-absent        Adm-clerical        Unmarried   White   

<ipython-input-26-43cc6d7c5bb3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[[new_col for new_col in unique_splits]] = np.nan
/home/aimwps/anaconda3/envs/AIMwpS/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-26-43cc6d7c5bb3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

...

 Own-child
 Unmarried
 Other-relative
       age          workclass  fnlwgt      education  education-num  \
24101   33   Self-emp-not-inc  127894    Prof-school             15   
31680   34            Private  102130        HS-grad              9   
32508   45            Private  155093           10th              6   
16875   53            Private  195638        HS-grad              9   
17789   42   Self-emp-not-inc  201908    Prof-school             15   
24301   48            Private   39530        HS-grad              9   
24338   48            Private  174533        HS-grad              9   
21042   27            Private  192936      Bachelors             13   
10687   33            Private  112383   Some-college             10   
13791   52   Self-emp-not-inc  217210        HS-grad              9   

            marital-status          occupation     relationship    race  \
24101   Married-civ-spouse      Prof-specialty          Husband   White   
31680   Married-civ-spouse    

<ipython-input-26-43cc6d7c5bb3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_col] = df[col_name].str.contains(new_col)
<ipython-input-26-43cc6d7c5bb3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_col] = df[new_col].map({True: 1, False: 0})


 Cambodia
 Thailand
 Ecuador
 Laos
 Taiwan
 Haiti
 Portugal
 Dominican-Republic
 El-Salvador
 France
 Guatemala
 Italy
 China
 South
 Japan
 Yugoslavia
 Peru
 Outlying-US(Guam-USVI-etc)
 Scotland
 Trinadad&Tobago
 Greece
 Nicaragua
 Vietnam
 Hong
 Ireland
 Hungary
 Holand-Netherlands


/home/aimwps/anaconda3/envs/AIMwpS/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


       age          workclass  fnlwgt      education  education-num  \
199     24   Self-emp-not-inc   32921        HS-grad              9   
17067   23            Private  193586        HS-grad              9   
32552   43            Private   84661      Assoc-voc             11   
19025   47            Private  155659   Some-college             10   
1517    59            Private  284834      Bachelors             13   
5051    52            Private   99185        HS-grad              9   
14656   38            Private  193815        Masters             14   
29417   23            Private  208946      Bachelors             13   
12168   38        Federal-gov  214542   Some-college             10   
1496    32            Private   80058           11th              7   

            marital-status          occupation    relationship    race  \
199          Never-married               Sales   Not-in-family   White   
17067   Married-civ-spouse        Adm-clerical            Wife   White

In [ ]:
#train / test... you know the drill
from sklearn.model_selection import train_test_split
from sklearn import svm

adult_df = pd.read_csv("data/adult_clean_ehc.csv")

Y = adult_df['target'].replace({" <=50K": 0, " >50K": 1})
print(Y)

X = adult_df.drop(['target','fnlwgt','education-num','marital-status','education','workclass','relationship','occupation','sex','race','relationship','native-country'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(X,Y, random_state=909)
svc = svm.SVC(random_state=909).fit(x_train, y_train)
predict = svc.predict(x_test)
test_score = svc.score(x_test, y_test)
print(test_score)

# Discussion of results